## single family
* most single family homes should be L2
* some fraction L1? 10%
* 80% EV owners live in single family homes <- this is harder to code up
* 2035 60% of EV owners live in single family homes

* these numbers should come from the stock rollover model - registration data?

## multi-unit 
* some residents wont have access to charging
* 20% have L2 at home (increase to 75% by 2035) should also charge while out. 
* 25% work L2 (increase to 50% by 2035) and public but not home
* remaining % public DCFC/L2 everyone has access to this. 

## workplace 
* what % have access at work?
* default to 10% have access to work charging, will increase over time

* 50% of civic buildings have charging
* 10% of industrial 
* TNCs are either L2 or DCFC

* use destination_building_use_l2

## Public 
* Higher L2 = 19 kw
* while shopping or eating dcfc
* if retail - DCFC
* if open parking - L2
* if trip longer than 100 miles, assume next charge is DCFC
* the lower limit of dwell time should be longer for L2

* what about light duty fleet vehicles? -> ask Dave


## What outputs?
* number of chargers per type per county. 
* load curve for each charger type per county. 

# TODO: (Friday EOD)
### doesn't have to be complete but needs to be coherent. 
* implement the single vs multi housing. 
    * make it probablistic using np.random.choice with the power level. 
    * future: use real distributions
* add some public L2
* set work on 20% rest on 0%

* worry about the rest later.
* write the code to make the outputs. this should really be number one. needs to work by tomorrow eve or else nothing matters. 

* need a percent chance on either l2 v none or l2 v dcfc. 

In [119]:
import pandas as pd
import numpy as np
import replicaEVSE.load_curve as sim
import os
import joblib
import dask.dataframe as dd

%reload_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

datadir = '../../data/'
# datadir = '../../../data'

mode = 'PRIVATE_AUTO'
simulation_id = 'dev'
test = True

#Created in the EIA_data_download.ipynb notebook
#existing_load=pd.read_csv('../../data/EIA_demand_summary.csv') 
existing_load=pd.read_csv(os.path.join(datadir,'EIA_demand_summary.csv'))
if test:
    df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_subsample.parquet'))
    df = df.loc[df['mode'] == mode]
    df = df.sort_values(by=['person_id', 'weekday', 'start_time']).reset_index(drop=True)
    simulation_id = 'dev'

    ddf = dd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

else: 


    # read in the joined trips and population data sets
    merged_df = pd.read_parquet(os.path.join(datadir, 'wa_pop_and_trips_sorted.parquet'))

    # right now, only look at private auto trips
    df = merged_df.loc[merged_df['mode'] == mode]

Note: Using this subset of the data means we cant see all the trips taken by a person_id since it wasn't sorted before this. 

In [79]:
df

,index,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
0,4199,7952205927434267893,10000366922410938523,PRIVATE_AUTO,HOME,EAT,0 days 12:41:15,0 days 12:43:31,0.602554,nan,530330068002,47.636260,-122.358964,530330067004,47.635745,-122.347078,commercial,retail,commercial,retail,residential,multi_family,residential,multi_family,47.638249,-122.356634,47.636388,-122.346274,thursday,1732374551127832797,530330067004,nan,nan,53033006700,nan,nan,35_64,62.0,M,white,not_hispanic_or_latino,20000_40000,22176.0,notinlf,some_college,not_attending_school,naics72,head_of_household,0,nan,not_working,renter,same_house,5_person,40000_75000,59308.0,nonfamily_single,1,multiple_units,core,english,47.636388,-122.346274,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
1,3575,506829784905170377,10001830118790204821,PRIVATE_AUTO,HOME,EAT,0 days 19:43:00,0 days 19:56:05,1.011157,nan,530610516012,47.844719,-122.299453,530610516021,47.834790,-122.308797,commercial,retail,commercial,retail,mixed_use,single_family,residential,single_family,47.840838,-122.300091,47.834503,-122.313617,thursday,3635700334446387909,530610516021,530350805002,530610516014,53061051602,53035080500,53061051601,18_24,18.0,M,white,not_hispanic_or_latino,0_20000,216.0,employed,high_school,school,naics722513,child,0,nan,carpool,renter,same_house,3_person,40000_75000,41273.0,family_single,2,multiple_units,core,english,47.834503,-122.313617,NaN,NaN,47.845652,-122.315406,worked_in_person,HOME
2,4816,13722390883687412311,10002290586277908152,PRIVATE_AUTO,HOME,EAT,0 days 18:40:12,0 days 18:46:32,2.150550,nan,530330323092,47.675673,-122.126007,530330228023,47.663252,-122.138520,mixed_use,office,commercial,retail,residential,single_family,residential,single_family,47.670172,-122.111026,47.664865,-122.135813,thursday,4129290245843427363,530330228023,nan,nan,53033022802,nan,nan,25_34,34.0,F,asian,not_hispanic_or_latino,lte_0,0.0,notinlf,advanced_degree,not_attending_school,not_working,spouse,0,nan,not_working,owner,same_house,5_person,125000_plus,319710.0,married_couple,3_plus,single_family,core,asian_pacific,47.664865,-122.135813,NaN,NaN,NaN,NaN,under_16_or_notinlf,HOME
3,247,11325025596515485190,10002759413662457525,PRIVATE_AUTO,WORK,HOME,0 days 07:24:41,0 days 07:37:33,6.867275,nan,530670109103,47.016462,-122.909304,530670112003,47.047701,-122.808723,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,47.016530,-122.908200,47.040000,-122.800000,thursday,3706854826325843051,530670109103,530670112003,nan,53067010910,53067011200,nan,25_34,33.0,F,white,not_hispanic_or_latino,40000_80000,51862.0,employed,high_school,not_attending_school,naics92,spouse,0,nan,driving,owner,same_house,4_person,75000_125000,111288.0,married_couple,3_plus,single_family,core,other,47.016530,-122.908200,47.040000,-122.80000,NaN,NaN,worked_in_person,WORK
4,4243,2725037000145676387,10002759413662457525,PRIVATE_AUTO,SHOP,WORK,0 days 09:52:13,0 days 09:58:59,1.866797,nan,530670112003,47.047701,-122.808723,530670112003,47.047701,-122.808723,civic_institutional,education,civic_institutional,edu

In [29]:
df[df['weekday'] == 'thursday'].person_id.value_counts()

12464838112032829470    4
3739083454226448410     4
6843763911659648578     4
8653789151746634547     4
16894664332853945147    4
                       ..
15814387655803398606    1
15813459731647938179    1
15812214865397043243    1
15809945927865456060    1
9999649577096558870     1
Name: person_id, Length: 2517, dtype: int64

In [54]:
df.travel_purpose.value_counts()

HOME                   1806
SHOP                   1247
EAT                     595
WORK                    515
SOCIAL                  415
RECREATION              240
MAINTENANCE             181
OTHER_ACTIVITY_TYPE      98
WORK_FROM_HOME           60
SCHOOL                   53
LODGING                  17
Name: travel_purpose, dtype: int64

In [30]:
df['destination_building_use_l2'].value_counts()

single_family               1889
retail                      1839
office                       392
multi_family                 365
education                    183
non_retail_attraction        155
open_space                   105
industrial                   102
civic_institutional           81
healthcare                    55
transportation_utilities      46
agriculture                    8
other                          4
unknown                        3
Name: destination_building_use_l2, dtype: int64

In [106]:
df['destination_building_use_l2'].unique()

array(['multi_family', 'single_family', 'education', 'office', 'retail',
       'civic_institutional', 'non_retail_attraction', 'open_space',
       'industrial', 'transportation_utilities', 'healthcare',
       'agriculture', 'other', 'unknown'], dtype=object)

In [37]:
trips_df = df[df['person_id'] == '3739083454226448410'].copy()
trips_df[['charge_type', 'weekday', 'start_time', 'end_time', 'distance_miles', 'destination_building_use_l2']]

,charge_type,weekday,start_time,end_time,distance_miles,destination_building_use_l2
3296,HOME,saturday,0 days 10:12:58,0 days 10:24:18,3.722588,single_family
3297,PUBLIC,thursday,0 days 09:06:41,0 days 09:12:16,2.273954,office
3298,HOME,thursday,0 days 12:26:09,0 days 12:30:57,1.549859,single_family
3299,PUBLIC,thursday,0 days 16:32:01,0 days 16:38:50,2.599557,non_retail_attraction
3300,PUBLIC,thursday,0 days 16:59:47,0 days 17:11:20,4.968752,retail


In [47]:
trips_df = df[df['person_id'] == '6843763911659648578'].copy()
trips_df[['charge_type', 'weekday', 'start_time', 'end_time', 'distance_miles', 'destination_building_use_l2']]

,charge_type,weekday,start_time,end_time,distance_miles,destination_building_use_l2
4236,PUBLIC,thursday,0 days 11:44:27,0 days 13:13:24,59.672562,single_family
4237,PUBLIC,thursday,0 days 13:32:57,0 days 14:15:21,12.340126,education
4238,PUBLIC,thursday,0 days 15:59:21,0 days 16:05:30,2.341793,retail
4239,PUBLIC,thursday,0 days 16:13:59,0 days 16:25:30,6.274836,retail


# load full data set to get better picture of this persons day

In [61]:
trips_ddf = ddf[(ddf['person_id'] == '6843763911659648578') & (ddf['weekday'] == 'thursday')]
snapshot = trips_ddf.compute()

### add stop duration

In [62]:
trips = snapshot
trips['stop_duration'] = 0
for i in range(0, len(trips)-1):
    trips.iloc[i, trips.columns.get_loc('stop_duration')] =\
        trips.iloc[i+1, trips.columns.get_loc('start_time')] -\
        trips.iloc[i, trips.columns.get_loc('end_time')]
    trips.iloc[len(trips)-1, trips.columns.get_loc('stop_duration')] =\
        trips.iloc[0, trips.columns.get_loc('start_time')] -\
        (trips.iloc[len(trips)-1, trips.columns.get_loc('end_time')
                    ]-pd.to_timedelta('1 day'))

In [66]:
trips.columns

Index(['activity_id', 'person_id', 'mode', 'travel_purpose',
       'previous_activity_type', 'start_time', 'end_time', 'distance_miles',
       'vehicle_type', 'origin_bgrp', 'origin_bgrp_lat', 'origin_bgrp_lng',
       'destination_bgrp', 'destination_bgrp_lat', 'destination_bgrp_lng',
       'origin_land_use_l1', 'origin_land_use_l2', 'origin_building_use_l1',
       'origin_building_use_l2', 'destination_land_use_l1',
       'destination_land_use_l2', 'destination_building_use_l1',
       'destination_building_use_l2', 'origin_lat', 'origin_lng',
       'destination_lat', 'destination_lng', 'weekday', 'household_id',
       'BLOCKGROUP', 'BLOCKGROUP_work', 'BLOCKGROUP_school', 'TRACT',
       'TRACT_work', 'TRACT_school', 'age_group', 'age', 'sex', 'race',
       'ethnicity', 'individual_income_group', 'individual_income',
       'employment', 'education', 'school_grade_attending', 'industry',
       'household_role', 'subfamily_number', 'subfamily_relationship',
       'commute_mo

## all the trips this person took in a day

In [63]:
cols = ['person_id', 'charge_type', 'travel_purpose', 'destination_building_use_l2', 'weekday', 'start_time', 'end_time', 'stop_duration', 'distance_miles', ]
trips[cols]

,person_id,charge_type,destination_building_use_l2,weekday,start_time,end_time,stop_duration,distance_miles
8535,6843763911659648578,PUBLIC,single_family,thursday,0 days 11:44:27,0 days 13:13:24,0 days 00:19:33,59.672562
8536,6843763911659648578,PUBLIC,education,thursday,0 days 13:32:57,0 days 14:15:21,0 days 00:08:49,12.340126
8539,6843763911659648578,PUBLIC,education,thursday,0 days 14:24:10,0 days 14:24:10,0 days 00:09:00,0.000000
8540,6843763911659648578,PUBLIC,retail,thursday,0 days 14:33:10,0 days 15:18:50,0 days 00:40:31,33.453654
8543,6843763911659648578,PUBLIC,retail,thursday,0 days 15:59:21,0 days 16:05:30,0 days 00:08:29,2.341793
8544,6843763911659648578,PUBLIC,retail,thursday,0 days 16:13:59,0 days 16:25:30,0 days 00:16:07,6.274836
8546,6843763911659648578,HOME,single_family,thursday,0 days 16:41:37,0 days 17:01:56,0 days 18:42:31,12.988673


In [85]:
# trips_df = df[df['person_id'] == '3739083454226448410'].copy()
trips_df = trips.copy()
# Create charge_type column from travel_purpose column
trips_df['charge_type'] = trips_df.travel_purpose.copy()
trips_df.loc[trips_df.charge_type.isin(
    ['WORK', 'HOME']) == False, 'charge_type'] = 'PUBLIC'

trips_list = []
loads_collection = []
for j in list(set(trips_df.person_id)):
    # Get the subset of trips made by person j
    trips_temp = trips_df.loc[trips_df.person_id == j].copy()
    # Get the person data for person j
    # person_temp = persons_df.loc[persons_df.person_id == j].copy()
    person_temp = pd.DataFrame()
    # Determine vehicle energy consumpsion rate in kWh/mi
    # NOTE: this is currently a dummy function = 0.3
    vehicle_energy_consumption = sim.determine_energy_consumption(trips_temp)

    charge_dfs = []
    for day in ['thursday', 'saturday']:
        if len(trips_temp.loc[trips_temp.weekday == day]) > 0:
            # For each day (thursday and saturday), get charger availability for person j
            # and determine which stopping events will result in charges
            charger_availability = determine_charger_availability(trips_temp.loc[trips_temp.weekday == day])
            print(charger_availability, day)

{'HOME': 7.2, 'PUBLIC': 150} thursday


In [86]:
sim.create_charging_events(
    df_trips=trips_temp.loc[trips_temp.weekday == 'thursday'].copy(),
    charger_availability={'HOME': 7.2, 'PUBLIC': 150},
    consumption_kWh_per_mi=0.3,
    weekday='thursday'
)


,activity_id,charger_power_kW,stop_duration,charge_energy_used_kWh,charge_opportunity_remaining_kWh
8535,12615986329685930368,150.0,0 days 00:19:33,0.000000,48.875000
8540,6957766517856863131,150.0,0 days 00:40:31,0.000000,101.291667
8544,12808269348753414334,150.0,0 days 00:16:07,0.000000,40.291667
8546,12602737506276955197,7.2,0 days 18:42:31,33.716917,100.985083


In [71]:
weekday = 'thursday'
charger_availability = determine_charger_availability(trips_temp.loc[trips_temp.weekday == weekday])
trips_temp.loc[trips_temp.weekday == weekday].charge_type, charger_availability

(8535    PUBLIC
 8536    PUBLIC
 8539    PUBLIC
 8540    PUBLIC
 8543    PUBLIC
 8544    PUBLIC
 8546      HOME
 Name: charge_type, dtype: object,
 {'HOME': 7.2, 'PUBLIC': 150})

In [16]:
def determine_charger_availability(
    person_id
    trips_df,
    frac_work_charging,
    frac_multiunit_charging=0.2,
    frac_singleunit_charging=1.0,
    frac_public_dcfc=0.9,
):
    """Depricated for now. Original version from John. Uses person data and trips data to determine 
    which types of charging the driver has access to, and what the order of preference is. 

    Parameters
    ----------
    trips_df : pandas DataFrame
        Trip data for one individual

    Returns
    -------
    dictionary
        Dictionary of available charging locations (HOME, WORK, and/or PUBLIC), with the power in kW for each option.
          The order of appearance in the dictionary sets the order of preference for the options.
    """
    # Notes
    # This should be revised to incorporate information about the person.
    # For example, the location of the home/work and type of home/work should inform whether
    # charging is available at these locations.
    # Also, some drivers may have different orders of preference for the
    # different charging options.

    charge_set = list(set(trips_df.charge_type))
    charge_dict = {}
    
    # Before adding the options to the dictionary, check if the stop type exists in the trip data
    #  there are ranges over level 2, and DCFC are all public charging options
    if 'HOME' in charge_set:

        # all homeowners of single fam have L2
        charge_dict.update({'single_fam_HOME': 7.2})
    
    if 'WORK' in charge_set:
        # use random choice to determine if worker can charge at work
        frac_work_charging = 0.2
        work_charge = np.random.choice([7.2, 0], p=[frac_work_charging, 1-frac_work_charging])
        charge_dict.update({'WORK': work_charge})
    
    if 'PUBLIC' in charge_set:
        charge_dict.update({'PUBLIC': 150})
    return charge_dict

Need to look at two columns to define charger type. 

In [97]:
person_id = '6843763911659648578'
person_temp = df.loc[df.person_id == person_id].copy()
person_temp

,index,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng,weekday,household_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh,charge_type
4236,2886,12615986329685930368,6843763911659648578,PRIVATE_AUTO,SOCIAL,HOME,0 days 11:44:27,0 days 13:13:24,59.672562,nan,530459611002,47.242080,-122.963878,530530730061,46.888716,-122.424464,residential,single_family,residential,single_family,residential,single_family,residential,single_family,47.217109,-122.977100,46.918015,-122.517207,thursday,3078205589430204724,530459611002,nan,nan,53045961100,nan,nan,15_17,17.0,F,white,hispanic_or_latino,lte_0,0.0,notinlf,k_12,school,not_working,child,0,nan,not_working,owner,same_house,6_person,10000_40000,39799.0,married_couple,3_plus,single_family,core,spanish,47.217109,-122.9771,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC
4237,3816,18445877218022607594,6843763911659648578,PRIVATE_AUTO,OTHER_ACTIVITY_TYPE,SOCIAL,0 days 13:32:57,0 days 14:15:21,12.340126,nan,530530730061,46.888716,-122.424464,530670123202,47.013010,-122.678628,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,46.918015,-122.517207,47.003885,-122.670146,thursday,3078205589430204724,530459611002,nan,nan,53045961100,nan,nan,15_17,17.0,F,white,hispanic_or_latino,lte_0,0.0,notinlf,k_12,school,not_working,child,0,nan,not_working,owner,same_house,6_person,10000_40000,39799.0,married_couple,3_plus,single_family,core,spanish,47.217109,-122.9771,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC
4238,4751,6478052449202763131,6843763911659648578,PRIVATE_AUTO,SHOP,EAT,0 days 15:59:21,0 days 16:05:30,2.341793,nan,530459608003,47.228712,-123.071527,530459606001,47.226641,-123.173361,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.213593,-123.099694,47.233347,-123.125322,thursday,3078205589430204724,530459611002,nan,nan,53045961100,nan,nan,15_17,17.0,F,white,hispanic_or_latino,lte_0,0.0,notinlf,k_12,school,not_working,child,0,nan,not_working,owner,same_house,6_person,10000_40000,39799.0,married_couple,3_plus,single_family,core,spanish,47.217109,-122.9771,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC
4239,5073,12808269348753414334,6843763911659648578,PRIVATE_AUTO,SHOP,SHOP,0 days 16:13:59,0 days 16:25:30,6.274836,nan,530459606001,47.226641,-123.173361,530459609001,47.200883,-123.086679,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.233347,-123.125322,47.201051,-123.091121,thursday,3078205589430204724,530459611002,nan,nan,53045961100,nan,nan,15_17,17.0,F,white,hispanic_or_latino,lte_0,0.0,notinlf,k_12,school,not_working,child,0,nan,not_working,owner,same_house,6_person,10000_40000,39799.0,married_couple,3_plus,single_family,core,spanish,47.217109,-122.9771,NaN,NaN,NaN,NaN,under_16_or_notinlf,PUBLIC


In [112]:
df['travel_purpose'].unique()

array(['HOME', 'WORK', 'SHOP', 'SCHOOL', 'SOCIAL', 'MAINTENANCE',
       'RECREATION', 'EAT', 'OTHER_ACTIVITY_TYPE', 'WORK_FROM_HOME',
       'LODGING'], dtype=object)

In [130]:
person_id = '6843763911659648578'
# person_temp = df.loc[df.person_id == person_id].copy()

destinations = ['multi_family', 'single_family', 'education', 'office', 'retail',
       'civic_institutional', 'non_retail_attraction', 'open_space',
       'industrial', 'transportation_utilities', 'healthcare',
       'agriculture', 'other', 'unknown']

# Define a function to map the values
def map_charge_type(row):
    if row['travel_purpose'] == 'HOME':
        if row['building_type'] == 'single_family':
            return 'single_family_home'
        else:
            return 'multi_family_home'
    else:
        return 'public'
# Apply the mapping function to create the charge_type column
df['charge_type'] = df.apply(map_charge_type, axis=1)

cols = cols = ['person_id', 'charge_type', 'travel_purpose', 'building_type', 'destination_building_use_l2', 'weekday', 'start_time', 'end_time', 'distance_miles', ]
df[cols]


,person_id,charge_type,travel_purpose,destination_building_use_l2,weekday,start_time,end_time,distance_miles
0,10000366922410938523,multi_family_home,HOME,multi_family,thursday,0 days 12:41:15,0 days 12:43:31,0.602554
1,10001830118790204821,multi_family_home,HOME,single_family,thursday,0 days 19:43:00,0 days 19:56:05,1.011157
2,10002290586277908152,single_family_home,HOME,single_family,thursday,0 days 18:40:12,0 days 18:46:32,2.150550
3,10002759413662457525,public,WORK,education,thursday,0 days 07:24:41,0 days 07:37:33,6.867275
4,10002759413662457525,public,SHOP,office,thursday,0 days 09:52:13,0 days 09:58:59,1.866797
...,...,...,...,...,...,...,...,...
5222,999052029215570835,public,EAT,retail,saturday,0 days 14:46:37,0 days 15:00:32,6.035001
5223,999052029215570835,public,EAT,retail,thursday,0 days 14:31:55,0 days 14:47:28,6.282163
5224,9997154344201549171,multi_family_home,HOME,single_family,thursday,0 days 13:14:29,0 days 13:24:48,2.893042
5225,9999649577096558870,public,SOCIAL,multi_family,saturday,0 days 11:58:34,0 days 12:14:46,1.536068


In [129]:
df['building_type'].value_counts()

single_family     3814
multiple_units     702
several_units      254
GQ_structure       106
Name: building_type, dtype: int64

In [ ]:
import pandas as pd

# Create a sample DataFrame with the housing_type and travel_purpose columns
df = pd.DataFrame({
    'housing_type': ['single_family', 'multiple_units', 'several_units'],
    'travel_purpose': ['HOME', 'WORK', 'HOME']
})

# Define a function to map the values based on the conditions
def map_charge_type(row):
    if row['travel_purpose'] == 'HOME':
        if row['housing_type'] == 'single_family':
            return 'single_family_home'
        elif row['housing_type'] in ['multiple_units', 'several_units']:
            return 'multi_family_home'
    return None

# Apply the mapping function to create the charge_type column
df['charge_type'] = df.apply(map_charge_type, axis=1)

# Print the updated DataFrame
print(df)
